In [1]:


// a few dependencies for the notebook:
import $ivy.`com.github.pathikrit::better-files:3.9.1`
import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`
import $ivy.`com.lihaoyi::scalatags:0.12.0`

import better.files._

// load historia code

val jarpath = s"/home/bounder/target/scala-2.13/soot_hopper-assembly-0.1.jar"
assert(File(jarpath).exists, "must run jupyter notebook from docker")
interp.load.cp(os.Path(jarpath))

import $ivy.$                                         

import $ivy.$                             

import $ivy.$                              


import better.files._

// load historia code


jarpath: String = "/home/bounder/target/scala-2.13/soot_hopper-assembly-0.1.jar"

In [2]:



import edu.colorado.plv.bounder.symbolicexecutor.state.{InitialQuery,ReceiverNonNull}
import edu.colorado.plv.bounder.lifestate.LifeState

import edu.colorado.plv.bounder.{Driver,RunConfig, BounderUtil} // Historia utilities
import upickle.default.read
import upickle.default.write
import edu.colorado.plv.bounder.symbolicexecutor.state.{DisallowedCallin,NamedPureVar}
import edu.colorado.plv.bounder.lifestate.SAsyncTask
import edu.colorado.plv.bounder.PickleSpec
import edu.colorado.plv.bounder.lifestate.{AllMatchers,SJavaThreading,FragmentGetActivityNullSpec, LifeState, LifecycleSpec, RxJavaSpec, SAsyncTask, SDialog, SpecSignatures, SpecSpace, ViewSpec}
import edu.colorado.plv.bounder.lifestate.LifeState.{AbsMsg, LSSpec, LSTrue, Signature}
import edu.colorado.plv.bounder.symbolicexecutor.{PreciseApproxMode,LimitMsgCountDropStatePolicy}
import edu.colorado.plv.bounder.symbolicexecutor.Z3TimeoutBehavior

def getIsDocker():Boolean = {
    val res = BounderUtil.runCmdStdout("whoami")
    res.trim == "root"
}

val isDocker = getIsDocker()
// val isDocker = false //TODO: overridden

val historiaDir = if(isDocker) "/home/bounder" else "/Users/shawnmeier/Documents/source/historia/Historia/"

// define a function to call the JAR implementation of Historia with a configuration
// If changes are made to Historia, run "sbt compile" in the /home/implementation directory to regenerate the Historia JAR

def runHistoriaWithSpec(configPath:File, printThenDone:Boolean = false):String = {
    val javaMemLimit=20 // Gb Note that this only limits JVM not JNI which can go significantly higher
    val historiaJar = jarpath
    val apkRootDir = "/Users/shawnmeier/Documents/data/historia_generalizability"
    val outDir = configPath.parent.toString
    val config = read[RunConfig](configPath.contentAsString)
    val outSubdir = config.outFolder.get.replace("${baseDirOut}",outDir)
    val cmd = s"java -Xmx${javaMemLimit}G -jar ${historiaJar} -m verify -c ${configPath} -b ${apkRootDir} -u ${outDir} -o MEM --debug"
    
    if(printThenDone){
        println(cmd)
        cmd
    }else{
        //BounderUtil.runCmdStdout(cmd, Some("/Users/shawnmeier/software/z3/build"))
        BounderUtil.runCmdFileOut(cmd, configPath.parent).toString
    }
}

// def printOutput(

def runAndPrint(configPath:File, allSpecs:Iterable[LSSpec], printThenDone:Boolean = false):String = {
    val res = runHistoriaWithSpec(configPath,printThenDone)
    println("run result")
    println(res)   
    println("specified messages")
    val msgSigs = allSpecs.flatMap(spec =>
                Set(spec.target) ++ spec.pred.allMsg).map(msg => msg.identitySignature)
    println(msgSigs)
    println(msgSigs.size)
    res
}

val v = NamedPureVar("v")
val a = NamedPureVar("a")
val f = NamedPureVar("f")

import edu.colorado.plv.bounder.symbolicexecutor.state.{InitialQuery,ReceiverNonNull}

import edu.colorado.plv.bounder.lifestate.LifeState


import edu.colorado.plv.bounder.{Driver,RunConfig, BounderUtil} // Historia utilities

import upickle.default.read

import upickle.default.write

import edu.colorado.plv.bounder.symbolicexecutor.state.{DisallowedCallin,NamedPureVar}

import edu.colorado.plv.bounder.lifestate.SAsyncTask

import edu.colorado.plv.bounder.PickleSpec

import edu.colorado.plv.bounder.lifestate.{AllMatchers,SJavaThreading,FragmentGetActivityNullSpec, LifeState, LifecycleSpec, RxJavaSpec, SAsyncTask, SDialog, SpecSignatures, SpecSpace, ViewSpec}

import edu.colorado.plv.bounder.lifestate.LifeState.{AbsMsg, LSSpec, LSTrue, Signature}

import edu.colorado.plv.bounder.symbolicexecutor.{PreciseApproxMode,LimitMsgCountDropStatePolicy}

import edu.colorado.plv.bounder.symbolicexecutor.Z3TimeoutBehavior


defined function getIsDocker
isDocker: Boolean = true
historiaDir: String 

Row 2 ap ex
-----------


In [ ]:


val row2_outputDir = File(s"/home/notebooks/Row2_Antennapod_execute_bug/WitStoriaOut")


def run2():Unit = {
    val outputDir = row2_outputDir
    val row2Specs = Set[LSSpec](
        ViewSpec.clickWhileNotDisabled,
        LifecycleSpec.Activity_createdOnlyFirst
      )



    val methodSignature = LifeState.Signature("com.example.row2antennapodexecute.RemoverActivity",
              "void onClick(android.view.View)")


    val initialQuery = DisallowedCallin(
              methodSignature.base,
              methodSignature.methodSignature,
              SAsyncTask.disallowDoubleExecute)
    // val initialQuery = ReceiverNonNull(
    //         methodSignature,
    //         34,
    //         None
    //     )

    val inputApk = s"${historiaDir}notebooks/Row2_Antennapod_execute_bug/app/build/outputs/apk/debug/app-debug.apk"
    val cfg =  RunConfig(apkPath = inputApk.toString, 
              outFolder = Some(outputDir.toString),
              initialQuery = List(initialQuery), truncateOut=false, 
                         specSet = PickleSpec(row2Specs, Set(SAsyncTask.disallowDoubleExecute), Set(ViewSpec.fv1)),
                         approxMode = PreciseApproxMode(true)
                        )
    
    val allSpecs = (row2Specs ++ Set[LSSpec](SAsyncTask.disallowDoubleExecute))


    val cfgPath = (outputDir / "cfg.json")
    println(cfgPath)
    cfgPath.overwrite(write(cfg))
    

    // val res = runHistoriaWithSpec(cfgPath)
    // println("withist history")
    // println(res.witnessExplanation)
    

    runAndPrint(cfgPath, allSpecs)
}

run2()

In [ ]:
println(read[Driver.LocResult]((row2_outputDir / "result_0.txt").contentAsString).witnessExplanation)

Row 1 
-----

In [3]:
val row1_outputDir = File("/home/notebooks/reachExpSmall/row1out")
def row1(){
    val methodSignature = LifeState.Signature("com.example.row1antennapodrxjava.ui.main.PlayerFragment",
              "void call(java.lang.Object)")
    
    val initialQuery = ReceiverNonNull(
            methodSignature,
            78, // line number in source code file
            None //Some(".*toString.*") // regular expression matching receiver (in case multiple dereferences on one line)
        )

    val row1Specs = Set(FragmentGetActivityNullSpec.getActivityNull,
        LifecycleSpec.Fragment_activityCreatedOnlyFirst,
        RxJavaSpec.call,
        LSSpec(f::Nil,Nil, SpecSignatures.Fragment_onCreate_entry,SpecSignatures.Fragment_onActivityCreated_entry)
      )

    val apkBase = if(isDocker) "/home/" else "/Users/shawnmeier/Documents/source/historia/Historia/"
    val inputApk = s"${apkBase}notebooks/AntennapodPlayerFragment_bug/app/build/outputs/apk/debug/app-debug.apk"
    val outputDir = row1_outputDir

    val cfgOutDir = if(!isDocker) row1_outputDir.toString.replace("/home/","/Users/shawnmeier/Documents/source/historia/Historia/") else row1_outputDir.toString
    val cfg =  RunConfig(apkPath = inputApk.toString, 
              outFolder = Some(cfgOutDir),
              initialQuery = List(initialQuery), truncateOut=false,
             specSet = PickleSpec(row1Specs),
             approxMode = PreciseApproxMode(true)
                        )
    
    val cfgPath = (outputDir / "cfg.json")
    cfgPath.overwrite(write(cfg))
    
    runAndPrint(cfgPath,row1Specs, false)
}
row1()

Not mac
run result
true
specified messages
HashSet(I_CIExit_RxJavasubscribe, I_CBEnter_FragmentonCreate, I_CBExit_FragmentonDestroy, I_CIExit_FragmentgetActivity, I_CIExit_rxJavaunsubscribe, I_CBEnter_FragmentonActivityCreated, I_CBEnter_rxJavacall)
7


row1_outputDir: File = /home/notebooks/reachExpSmall/row1out
defined function row1

In [4]:
println(read[Driver.LocResult]((row1_outputDir / "result_0.txt").contentAsString).witnessExplanation)

List(Future trace:
 CBEnter void onCreate(android.os.Bundle)( _T_,@2 )
CBEnter void onActivityCreated(android.os.Bundle)( _T_,@2 )
CIExit rx.Subscription subscribe(rx.functions.Action1)( @3,_T_,@2 )
CBExit void onDestroy()( _T_,@2 )
CBEnter void call(java.lang.Object)( _T_,@2 )
CIExit Activity getActivity()( NULL,@2 )
)


In [ ]:
File("/home/bounder/notebooks/AntennapodPlayerFragment_fix/").isDirectory

In [ ]:
row1_outputDir
